# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [4]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)
            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message

def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory(
        '/Users/sofia/Projects/workspace/courses/udemy/data-science-and-machine-learning-with-python/data/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory(
        '/Users/sofia/Projects/workspace/courses/udemy/data-science-and-machine-learning-with-python/data/emails/ham', 'ham'))

Let's have a look at that DataFrame:

In [5]:
data.head()

class  \
/Users/sofia/Projects/workspace/courses/udemy/d...  spam   
/Users/sofia/Projects/workspace/courses/udemy/d...  spam   
/Users/sofia/Projects/workspace/courses/udemy/d...  spam   
/Users/sofia/Projects/workspace/courses/udemy/d...  spam   
/Users/sofia/Projects/workspace/courses/udemy/d...  spam   

                                                                                              message  
/Users/sofia/Projects/workspace/courses/udemy/d...  <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Tr...  
/Users/sofia/Projects/workspace/courses/udemy/d...  1) Fight The Risk of Cancer!\n\nhttp://www.adc...  
/Users/sofia/Projects/workspace/courses/udemy/d...  1) Fight The Risk of Cancer!\n\nhttp://www.adc...  
/Users/sofia/Projects/workspace/courses/udemy/d...  ##############################################...  
/Users/sofia/Projects/workspace/courses/udemy/d...  I thought you might like these:\n\n1) Slim Dow...

Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [6]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values

classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [7]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='|S4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [11]:
import numpy as np

# split train and test sets
idx = np.random.rand(len(data)) < 0.8
train = data[idx]
test = data[~idx]
print "train set size: %s" % len(train)
print "test set size: %s" % len(test)
print ""

# fit classifier on train set
vectorizer = CountVectorizer()
train_counts = vectorizer.fit_transform(train['message'].values)
classifier = MultinomialNB()
train_targets = train['class'].values
classifier.fit(train_counts, train_targets)

# predict on test test
test_counts = vectorizer.transform(test['message'])
predictions = classifier.predict(test_counts)

# compute accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test['class'].values, predictions)
print "accuracy on test set: %s" % accuracy

train set size: 2374
test set size: 626

accuracy on test set: 0.950479233227
